In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from numpy.random import choice

In [34]:
# Declare parameters
#---------------------------------------
n_steps = 10001
nx = 200
T = 1
k = 0.5

n_cells = 1000

# Create initial grid
grid_0 = np.zeros((nx + 2, nx + 2))

# Give each cell a unique integer id
grid_0[1:-1, 1:-1] = np.random.choice(range(1, nx*nx+1), (nx,nx), replace=False)

for j in range(1, nx // 40):
    i = 0
    length = np.random.uniform(0.5 * nx // 40, 1.5 * nx // 40)

print(grid_0)
print(grid_0[10:20, 10:20])

[[    0.     0.     0. ...     0.     0.     0.]
 [    0.  3695. 10776. ... 29929.  2335.     0.]
 [    0. 12135. 38366. ... 22003. 39351.     0.]
 ...
 [    0.  5450.  4195. ...  7927. 19172.     0.]
 [    0. 34867. 30896. ... 29524.  4345.     0.]
 [    0.     0.     0. ...     0.     0.     0.]]
[[  882. 28485. 27525. 19477. 31229. 24726. 28969. 30683. 28852. 28947.]
 [29696.    85. 17208. 22052. 10154. 25913.  8699. 11467. 29537. 29328.]
 [ 7239.  7619. 17285. 25098. 15213. 20549.  5861. 11458.  9129.  1367.]
 [27038. 15736. 11383. 14307.  1191. 39531. 24888.  8254. 36281. 13241.]
 [39221. 39373.  6964. 26493. 16375.  3493. 37718. 10344.  8377. 38861.]
 [11691. 15655. 38012.  7895. 13011. 15087. 18579. 27001. 20816. 14363.]
 [ 2259.  6485. 21722.  6795. 17366. 16622.  3989. 14003.  6232.   471.]
 [30665.  3335. 38811. 28227. 38457. 28034. 33473. 20929. 14454. 14893.]
 [28542. 12493. 27014.  8384. 11142. 27884.  6895. 15555. 15770. 17726.]
 [29084.  2270. 15777.  4286.   211. 35986.

In [2]:
def kronecker_delta(i, j):
    return i == j

# def neighborhood_ids(grid, i, j):
#     return grid[i-1, j], grid[i+1, j], grid[i, j-1], grid[i, j+1]

In [120]:
class Cellular_Potts:
    def __init__(self, labda = 1, n_steps = 10001, nx = 200, T = 1, k = 1):
        self.n_steps, self.nx, self.T, self.k, self.labda = n_steps, nx, T, k, labda
        
        # Create initial grid
        self.grid_0 = np.zeros((nx + 2, nx + 2))
        
        cell_id = 1
        randoms = np.random.choice(range(1, nx*nx+1), nx*nx, replace=False)
        for j in range(1, 40-1):
            i = nx // 40
            while i < nx - nx // 40:
                length = int(np.random.uniform(1 * nx // 40, 2 * nx // 40))
                self.grid_0[j * nx // 40 : min(nx, (j + 1) * nx // 40), i : min(nx - nx // 40, i + length)] = randoms[cell_id]
                i += length
                cell_id += 1
                
        print(cell_id)

        # Give each cell a unique integer id
#         self.grid_0[1:-1, 1:-1] = np.random.choice(range(1, nx*nx+1), (nx,nx), replace=False)
        # Create grid
        self.grid = self.grid_0.copy()
        
    def cell_type(self, cell_id):
        # placeholder
        # idk wtf to do here
        if cell_id == 0:
            return 0

        return cell_id % 2 + 1

    def equilibrium_area(self, cell_type):
        # placeholder
        if cell_type == 0:
            return -10 # or something else negative
        return 40
    
    def cell_area(self, cell_id):
        return np.count_nonzero(self.grid == cell_id)
    
    def J(self, type1, type2):
        # placeholder
        return type1 + type2
    
    def Moore(self, i, j):
        return self.grid[i-1, j], self.grid[i+1, j], self.grid[i, j-1], self.grid[i, j+1]
    
    def Hamil_vol(self):
        sum_H = 0
        
        for cell_id in np.unique(self.grid):
            if cell_id != 0:
                sum_H += (self.cell_area(cell_id) - self.equilibrium_area(self.cell_type(cell_id)))
        
        return self.labda * sum_H
    
    def Hamil_bond(self):
        sum_H = 0
        
        for i, j in np.ndindex(self.grid.shape):
            if i == 0 or j == 0 or i == self.nx - 1 or j == self.nx - 1:
                break
                
            cell_id = self.grid[i, j]
            for cell_id2 in self.Moore(i, j):
                sum_H += (self.J(self.cell_type(cell_id), self.cell_type(cell_id2)) 
                          * (1 - kronecker_delta(cell_id, cell_id2)))
                
        return sum_H
    
    def Hamil_bond_local(self, coordinates):
        sum_H = 0
        
        for coord in coordinates:
            i, j = coord
            if i == 0 or j == 0 or i >= self.nx - 2 or j >= self.nx - 2:
                break
                
            cell_id = self.grid[i, j]
            for cell_id2 in self.Moore(i, j):
                sum_H += (self.J(self.cell_type(cell_id), self.cell_type(cell_id2)) 
                          * (1 - kronecker_delta(cell_id, cell_id2)))
                
        return sum_H
                
    def Hamiltonian(self):
        return self.Hamil_vol() + self.Hamil_bond()
        
    def step(self):
        # Create list to store Magnetization modulus
    #     M = np.zeros(n_steps + 1)
    
#         old_H = self.Hamiltonian()
        
        # Randomly choose lattice site 
#         M[step:] += abs(np.sum(grid))

        i, j = np.random.randint(1, self.nx, 2)
        old_id = self.grid[i, j]
        
        i2, j2 = ((i-1,j),(i+1,j),(i,j-1),(i,j+1))[np.random.choice(4)]
        old_H_bond = self.Hamil_bond_local(list(set([(i,j), (i+1,j), (i-1,j), (i,j+1), (i,j-1), (i2,j2), (i2-1,j2), (i2+1,j2), (i2,j2-1), (i2,j2+1)])))
        self.grid[i, j] = self.grid[i2, j2]
        new_id = self.grid[i, j]
        
        dH_vol = self.labda * (0 if (old_id == new_id or (old_id != 0 and new_id != 0)) else (-1 if new_id == 0 else 1))
        dH_bond = self.Hamil_bond_local(list(set([(i,j), (i+1,j), (i-1,j), (i,j+1), (i,j-1), (i2,j2), (i2-1,j2), (i2+1,j2), (i2,j2-1), (i2,j2+1)]))) - old_H_bond
        dH = dH_vol + dH_bond
                               
        # Hamiltonian
#         new_H = self.Hamiltonian()
        
        # Probability
        P = np.exp(-(dH) / (self.k * self.T))

        # Update grid
        if P < np.random.sample():
            self.grid[i, j] = old_id

    def run(self):
        i = 0
        for step in range(self.n_steps):
            i +=1
            if i % 1000 == 0:
                print(i, end='\r')
            self.step()
            
            if i % 100000 == 0:
                # Create figures        
                fig = plt.figure(facecolor='w', figsize=(12,4), dpi=80)

            #     ax = plt.subplot2grid((1, 3), (0, 0), colspan=2)
            #     ax.plot( M/(np.arange(1,n_steps+2)*(nx*nx)) , 'brown')
            #     ax.grid(b=True, which='major', c='k', alpha=0.2)
            #     ax.set_xlabel('n')
            #     ax.set_ylabel('<|M|>')
            #     ax.set_title('Magnetization modulus for J = %d' % J)

                ax = plt.subplot2grid((1, 3), (0, 2))
                ax.matshow(self.grid[1:-1, 1:-1], cmap = 'jet')
                ax.set_title('n = %i' % self.n_steps)
                ax.axis('off')

                plt.tight_layout()
                plt.show()


In [121]:
# for i in [-1, 0, 1]:
Cellular_Potts(n_steps=10000000).run()

1053


KeyboardInterrupt: 